In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import anndata
import pickle
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import tarfile
import community
#from community import community_louvain
#import community.community_louvain as community_louvain
import networkx as nx
from sklearn.neighbors import kneighbors_graph
from scipy import stats
import seaborn as sns
import collections
import scirpy as ir
import random
random.seed(10)
#https://icbi-lab.github.io/scirpy/latest/tutorials/tutorial_3k_tcr.html
# https://icbi-lab.github.io/scirpy-paper/wu2020.html

In [ ]:
#pip install python-igraph==0.9.11

In [ ]:
#!pip install numexpr==2.7.3

In [ ]:
#!pip list

In [ ]:
#!pip install scirpy

In [ ]:
#!pip install community

#!pip install community

#!pip install scanpy

#!pip install NumPy==1.22.3

#!pip install daal==2021.4.0

#!pip install Numba

#!pip list
#!pip install openpyxl==3.0.7

In [ ]:
gc.collect()

In [ ]:
meta_df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/scRNAseq_TCR_samples_meta_LS-13_f.xlsx')
meta_df.index = meta_df['Sample ID']

In [ ]:
filtered_combined = anndata.read_h5ad('/burg/iicd/users/ls3456/GVHD/adata_file/filtered_combined_all_PatientID_f.h5ad')

filtered_combined.obs_names_make_unique()

#filtered_combined.uns['log1p']["base"] = None
#sc.tl.rank_genes_groups(filtered_combined, groupby="leiden", method='wilcoxon', key_added = "wilcoxon", use_raw=False)
#sc.pl.rank_genes_groups(filtered_combined, n_genes=25, sharey=False, key="wilcoxon")

filtered_combined

In [ ]:
sc.pl.umap(filtered_combined,color="leiden_0.15")

In [ ]:
filtered_combined

In [ ]:
tcell_ann = filtered_combined[filtered_combined.obs["leiden_0.15"].isin(["T_cells"])]

#tcell_ann = tcell_ann[tcell_ann.obs["LPorIE"]=="Lamina Propria Cells"]

tcell_ann

In [ ]:
sc.pl.umap(tcell_ann, color= ['leiden_0.15'])

In [ ]:
#sc.pl.umap(tcell_ann, color=['CD3D','CD3E',"NCAM1","KIR2DL4","GNLY","NCR1","NKG7","PTPRC",'CD4','CD8A','FOXP3'])

In [ ]:
gc.collect()

In [ ]:
new_hub_colormap = [
    'darkslateblue', #0
    'cornflowerblue', #1
    'teal',#2
    'red',#3
    'blueviolet',#4
    'skyblue',#5
    'orchid',#6
    'yellowgreen',#7
    'palevioletred',#8
    'orange',#9
    'cadetblue',#10
    'limegreen',#11
    'cyan',#12
    'gold',#13
    'slategray',#14
    'olive',#15
    'blue',#16
    'linen',#17
    'mistyrose',#18
    'peru',#19
    'darkturquoise',#20
]

In [ ]:
#for meta in tcell_ann.obs.columns:
    #randomize=np.arange(len(tcell_ann.obs[meta]))
    #np.random.shuffle(randomize)
    #fig, ax = plt.subplots(figsize=(8,8))
    #sns.set_style("whitegrid", {'axes.grid' : False})
    #colors = []
    #for _ in range(len(tcell_ann.obs[meta].unique())):
        #color = generate_random_color()
        #colors.append(color)
    #scatter=sns.scatterplot(x=tcell_ann.obsm["X_umap"][:,0][randomize],y=tcell_ann.obsm["X_umap"][:,1][randomize], hue=tcell_ann.obs[meta][randomize],s=2, palette= colors).set(title=meta)
    #plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
for meta in tcell_ann.obs.columns:
    randomize=np.arange(len(tcell_ann.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(8,8))
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=tcell_ann.obsm["X_umap"][:,0][randomize],y=tcell_ann.obsm["X_umap"][:,1][randomize], hue=tcell_ann.obs[meta][randomize],s=2).set(title=meta)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sc.pp.neighbors(tcell_ann, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)
sc.tl.umap(tcell_ann)

In [ ]:
tcell_ann.obsm['X_scVI'].shape

In [ ]:
sc.pl.umap(tcell_ann, color=['CD3D','CD3E','CD3G',"NCAM1","KIR2DL4","GNLY","NCR1","NKG7","PTPRC",'CD4','CD8A','FOXP3'])

In [ ]:
for meta in tcell_ann.obs.columns:
    randomize=np.arange(len(tcell_ann.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(8,8))
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=tcell_ann.obsm["X_umap"][:,0][randomize],y=tcell_ann.obsm["X_umap"][:,1][randomize], hue=tcell_ann.obs[meta][randomize],s=2).set(title=meta)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sc.pl.umap(tcell_ann, color=['leiden_0.15'])

In [ ]:
tcell_ann.uns['log1p']["base"] = None

In [ ]:
tcell_ann

In [ ]:
sc.set_figure_params(dpi=200)
#sc.pl.umap(tcell_ann_concat,color = 'leiden')
sc.pl.umap(tcell_ann,color = 'leiden',legend_loc = 'on data',legend_fontsize = 'xx-small')

In [ ]:
sc.tl.leiden(tcell_ann,resolution=0.3, key_added= "leiden")#,resolution=0.1,

sc.pl.umap(tcell_ann,color = 'leiden',legend_loc = 'on data',legend_fontsize = 'xx-small')

# Doublet detection again

In [ ]:
import numpy as np
import pandas as pd
#from matplotlib_venn import venn2

import scanpy as sc

from tqdm import tqdm
from glob import glob
import anndata as ad
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import tarfile
#import community.community_louvain as community_louvain
import networkx as nx
from sklearn.neighbors import kneighbors_graph
from scipy import stats
import seaborn as sns
import collections
#import scirpy as ir
import random
from scipy.stats import gaussian_kde

In [ ]:
leidenlst  = [i for i in range(7)]

In [ ]:
def view_doublets_bycluster(Tcell_markers,BCell_markers,namex,namey,tcell_ann_concat):
    num_cols = 4
    fig, ax = plt.subplots(7, 4, figsize=(20,25), gridspec_kw={'wspace':0.1},layout = 'constrained')

    for i, leiden in enumerate(leidenlst):
        row = i // num_cols
        col = i % num_cols
        subTcell = tcell_ann_concat[tcell_ann_concat.obs.leiden == str(leiden)]
        x = subTcell[:,'CD3D'].X.toarray()

        x_all =  np.zeros([np.shape(x)[0],len(Tcell_markers)])
        for j in np.arange(len(Tcell_markers)):
            x_all[:,j] = subTcell[:,Tcell_markers[j]].X.toarray()[:,0]
        x = np.mean(x_all,axis =1)


        y_all = np.zeros([np.shape(x)[0],len(BCell_markers)])
        for j in np.arange(len(BCell_markers)):
            y_all[:,j] = subTcell[:,BCell_markers[j]].X.toarray()[:,0]

        y = np.mean(y_all,axis=1)
        xy = np.vstack([x.T,y.T])
        try:
            z = gaussian_kde(xy)(xy)
            #ax = ax[row][col]
            im=ax[row][col].scatter(x,y, c=z, s=10)
            ax[row][col].set_xlabel(namex)
            ax[row][col].set_ylabel(namey)
            ax[row][col].set_title(leiden)

            plt.colorbar(im)
        except:
            pass

    return

def doublet_lst(Tcell_markers,BCell_markers,namex,namey,tcell_ann_concat):
    x = tcell_ann_concat[:,'CD3D'].X.toarray()
    x_all = np.zeros([np.shape(x)[0],len(Tcell_markers)])
    y_all = np.zeros([np.shape(x)[0],len(BCell_markers)])
    for j in np.arange(len(BCell_markers)):
        y_all[:,j] = tcell_ann_concat[:,BCell_markers[j]].X.toarray()[:,0]
    for j in np.arange(len(Tcell_markers)):
        x_all[:,j] =tcell_ann_concat[:,Tcell_markers[j]].X.toarray()[:,0]
    x = np.mean(x_all,axis =1)
    y = np.mean(y_all,axis=1)
    xy = np.vstack([x.T,y.T])

    z = gaussian_kde(xy)(xy)
    fig, ax = plt.subplots()
    im=ax.scatter(x,y, c=z, s=10)
    #ax.scatter(x,y, c=z, s=10)
    ax.set_xlabel(namex)
    ax.set_ylabel(namey)
    plt.colorbar(im)
    B_doublet_idx = y>0
    print(sum(B_doublet_idx))
    return B_doublet_idx

In [ ]:
# Bcell & Tcell doublets_ls_GOOD
# Studies have found 'CD79A' on T cells
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
BCell_markers = ['CD19','MS4A1']
namex = 'T cell'
namey = 'B cell'
view_doublets_bycluster(Tcell_markers,BCell_markers,namex,namey,tcell_ann)
B_doublet_idx = doublet_lst(Tcell_markers,BCell_markers,namex,namey,tcell_ann)

In [ ]:
# Myeloid & Tcell doublets_ls_GOOD
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
Myeloid = ['CD68','APOE','CD163','CD1C','CD33','CSF1R','MERTK']
namex = 'T cell'
namey = 'Myeloid'
view_doublets_bycluster(Tcell_markers,Myeloid,namex,namey,tcell_ann)
Myeloid_doublet_idx = doublet_lst(Tcell_markers,Myeloid,namex,namey,tcell_ann)

In [ ]:
# macrophage & T_ls_good
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
Macrophage = ['CD68','MRC1','MSR1','NRP1']#,'CD86'
namex = 'T cell'
namey = 'Macrophage'
view_doublets_bycluster(Tcell_markers,Macrophage,namex,namey,tcell_ann)
macrophage_doublet_idx = doublet_lst(Tcell_markers,Macrophage,namex,namey,tcell_ann)

In [ ]:
#nomocyte & t_ls_good
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
Monocyte = ['CD33','LYZ','FCN1','CSF3R']
namex = 'T cell'
namey = 'Monocyte'
view_doublets_bycluster(Tcell_markers,Monocyte,namex,namey,tcell_ann)
Monocyte_doublet_idx = doublet_lst(Tcell_markers,Monocyte,namex,namey,tcell_ann)

In [ ]:
# epithelial cell doublets_ls_good
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
epithelial_markers = [ 'VIL1' , 'CLDN3'  ,'OLFM4' ,'LGR5' ]
namex = 'T cell'
namey = 'epithelial cell'
view_doublets_bycluster(Tcell_markers,epithelial_markers,namex,namey,tcell_ann)
epithelial_doublet_idx = doublet_lst(Tcell_markers,epithelial_markers,namex,namey,tcell_ann)

In [ ]:
# Plasma cells_ MZB1 'MZB1'
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
plasma_markers2 = ['TNFRSF17', 'SDC1']
namex = 'T cell'
namey = 'plasma cell'
view_doublets_bycluster(Tcell_markers,plasma_markers2,namex,namey,tcell_ann)
plasma_doublet2_idx = doublet_lst(Tcell_markers,plasma_markers2,namex,namey,tcell_ann)

In [ ]:
# 'ACTA2'
Fibroblast_markers = ['ACTA2']
Tcell_markers = ['CD3E','CD3D','TRAC','TRBC1','TRBC2','TRDC']
namex = 'T cell'
namey = 'Fibroblast cell'
view_doublets_bycluster(Tcell_markers,Fibroblast_markers,namex,namey,tcell_ann)
Fibroblast_doublet_idx = doublet_lst(Tcell_markers,Fibroblast_markers,namex,namey,tcell_ann)

In [ ]:
tcell_ann.obs['B_doublet'] = B_doublet_idx
tcell_ann.obs['B_doublet'] = tcell_ann.obs['B_doublet'].astype(str)

tcell_ann.obs['Myeloid_doublet'] = Myeloid_doublet_idx
tcell_ann.obs['Myeloid_doublet'] = tcell_ann.obs['Myeloid_doublet'].astype(str)


tcell_ann.obs['Macrophage_doublet'] = macrophage_doublet_idx
tcell_ann.obs['Macrophage_doublet'] = tcell_ann.obs['Macrophage_doublet'].astype(str)

tcell_ann.obs['Monocyte_doublet'] = Monocyte_doublet_idx
tcell_ann.obs['Monocyte_doublet'] = tcell_ann.obs['Monocyte_doublet'].astype(str)

tcell_ann.obs['Epithelial_doublet'] = epithelial_doublet_idx
tcell_ann.obs['Epithelial_doublet'] = tcell_ann.obs['Epithelial_doublet'].astype(str)

tcell_ann.obs['Fibroblast_doublet'] = Fibroblast_doublet_idx
tcell_ann.obs['Fibroblast_doublet'] = tcell_ann.obs['Fibroblast_doublet'].astype(str)

# tcell_ann_concat.obs['plasma_doublet'] = plasma_doublet_idx
# tcell_ann_concat.obs['plasma_doublet'] = tcell_ann_concat.obs['plasma_doublet'].astype(str)


tcell_ann.obs['plasma_doublet2'] = plasma_doublet2_idx
tcell_ann.obs['plasma_doublet2'] = tcell_ann.obs['plasma_doublet2'].astype(str)

In [ ]:
def plot_doublet(tcell_ann,colorlabel):
# sc.pl.umap(tcell_ann_concat,color = 'B_doublet',
    #            palette={'False':'grey','True':'tab:red'})
    ax=sc.pl.umap(tcell_ann,color = colorlabel,groups=['True'],show=False)

    # We can change the 'NA' in the legend that represents all cells outside of the
    # specified groups
    legend_texts=ax.get_legend().get_texts()
    # Find legend object whose text is "NA" and change it
    for legend_text in legend_texts:
        if legend_text.get_text()=="NA":
            legend_text.set_text('False')

In [ ]:
plot_doublet(tcell_ann,'B_doublet')
plot_doublet(tcell_ann,'Myeloid_doublet')
plot_doublet(tcell_ann,'Macrophage_doublet')
plot_doublet(tcell_ann,'Monocyte_doublet')
plot_doublet(tcell_ann,'Epithelial_doublet')
plot_doublet(tcell_ann,'Fibroblast_doublet')
plot_doublet(tcell_ann,'plasma_doublet2')

In [ ]:
B_doublet_idx
Myeloid_doublet_idx
macrophage_doublet_idx
Monocyte_doublet_idx
epithelial_doublet_idx
#endothelial_doublet_idx
Fibroblast_doublet_idx 
plasma_doublet2_idx
result = [any([B_doublet_idx[i], Myeloid_doublet_idx[i], 
               macrophage_doublet_idx[i],Monocyte_doublet_idx[i],
                epithelial_doublet_idx[i],Fibroblast_doublet_idx[i],plasma_doublet2_idx[i]]) for i in range(len(B_doublet_idx))]

In [ ]:
tcell_ann.obs['if_doublet'] = result

In [ ]:
tcell_ann.obs['if_doublet'] = tcell_ann.obs['if_doublet'].astype(str)

In [ ]:
ax=sc.pl.umap(tcell_ann,color = 'if_doublet',groups=['True'],show=False)

# We can change the 'NA' in the legend that represents all cells outside of the
# specified groups
legend_texts=ax.get_legend().get_texts()
# Find legend object whose text is "NA" and change it
for legend_text in legend_texts:
    if legend_text.get_text()=="NA":
        legend_text.set_text('False')

In [ ]:
tcell_ann.obs['if_doublet']

In [ ]:
tcell_ann

In [ ]:
tcell_ann_sc = tcell_ann[tcell_ann.obs['if_doublet']== 'False']

In [ ]:
tcell_ann_sc

In [ ]:
tcell_ann_sc_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_sc.h5ad'
tcell_ann_sc.write(tcell_ann_sc_loc)

In [ ]:
tcell_ann_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann.h5ad'
tcell_ann.write(tcell_ann_loc)